# Projet 3
## Concevoir une application au service de la santé publique
L'objectif de ce projet est d'utiliser le jeu de données open source **Open Food Facts** afin de proposer une idée d'application en lien avec la nutrition.
Le jeu de données est téléchargeable à l'adresse:
https://world.openfoodfacts.org/data

Les variables sont définies à l'adresse:
https://world.openfoodfacts.org/data/data-fields.txt
### Partie 1 - Nettoyage du jeu de données

Dans cette première partie nous allons nettoyer le jeu de données afin d'éliminer les erreurs évidentes et de produire un jeu de données cleané, que nous exploiterons dans la partie deux.

In [1]:
# Importer les librairies
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

# Version des modules
print('Pandas:', pd.__version__)
print('Matplotlib:', matplotlib.__version__)
print('Numpy:', np.__version__)

Pandas: 1.0.5
Matplotlib: 3.2.2
Numpy: 1.18.5


Le fichier étant assez volumineux, on choisit de le lire par blocs de 10^4 lignes et d'évaluer le taux de remplissage des colonnes afin de ne charger que les colonnes les mieux remplies *(merci à Pierre-Antoine sur le forum pour l'astuce!)*.

In [2]:
# Solution pour charger uniquement les colonnes les mieux remplies :
# nombre de lignes à charger en même temps en mémoire
chunksize = 10 ** 4
filename = 'en.openfoodfacts.org.products.csv'
# calcul du taux de valeurs manquantes par bloc
nan_rates = []
for chunk in pd.read_csv(filename, chunksize=chunksize, sep='\t', low_memory=False):
    nan_rates.append(chunk.isna().mean(axis=0))
# moyenne des taux de remplissage des chunks
inds_nan_rate = pd.concat(nan_rates, axis=1).mean(axis=1)

#### Choix des variables à garder
**Données quantitatives:**
Dans la partie 2 du projet nous allons utliser un algorithme de clustering afin de montrer que les données nutritives suffisent à catégoriser les produits. A ce titre, plus nous avons de variables, meilleure sera la catégorisation. Cependant on se rend compte que lorsqu'on rentre trop dans le détail de la composition des produits (vitamines, minéraux...) les données deviennent très éparses et n'apportent pas nécessairement de plus-value. On choisit donc un seuil de remplissage des variables de 70%, ce qui devrait nous apporter un compromis correct entre utilité des variables et complexité du modèle.

**Alcool:**
Classiquement, on calcule l'énergie d'un produit en faisant la somme de la contribution de ses macronutriments: lipides, glucides, protéines. On oublie généralement l'alcool, qui apporte également son potentiel énergétique, car les produits mélangeant denrées alimentaires et alcool sont très minoritaires. On se propose ici de garder cette variable dans le calcul de l'énergie. De plus cette variable nous sera indispensable pour classifier correctement les boissons alcoolisées. Ainsi, bien que le taux de remplissage de cette colonne ne passe pas le seuil des 70% car elle concerne qu'une catégorie minoritaire de produits, nous choisissons de la garder car elle apporte des informations utiles.

**Fibres:**
De même, la variable sur les fibres est peu remplie car elle ne concerne que certaines catégories d'aliments mais reste utile, nous la gardons également.

**Données qualitatives:**
- Nutriscore: indice relatif à la qualité nutritionnelle d'un produit;
- Variables catégorielles: "pnns_groups", "nova_group", "main_category_en" sont des variables qui nous renseignent sur les catégories des produits et pourront nous être utiles;
- "brands", "countries_en" nous renseignent sur l'origine des produits.

**Métadonnées:**
On garde quelques données relatives à l'identification des produits, qui nous serons utiles à la fois pour identifier correctement les clusters et pour notre idée d'application.
- Code
- Url du produit / Url de la photo du produit
- Nom du produit

In [3]:
# Nom des colonnes ayant moins de 70% de valeurs manquantes
cols_to_load = inds_nan_rate[inds_nan_rate < 0.7].index.tolist()

# Certaines colonnes ne nous intéressent pas car redondantes ou non pertinentes
cols_to_discard = ['creator', 'created_t', 'created_datetime',
                    'last_modified_t', 'last_modified_datetime', 'brands_tags', 'categories', 
                    'categories_en', 'categories_tags','countries', 'countries_tags','states', 
                    'states_tags', 'ingredients_that_may_be_from_palm_oil_n',
                    'states_en','ingredients_text', 'main_category', 'image_url',
                    'image_ingredients_url', 'image_ingredients_small_url',
                    'image_nutrition_url', 'image_nutrition_small_url', 'nutrition-score-fr_100g']

# Colonnes à charger
cols_to_load = [col for col in cols_to_load if col not in cols_to_discard]

# Si les colonnes "fibres" et "alcool" ne sont pas remplies à plus de 30% elles sont rajoutées ici
if 'fiber_100g' not in cols_to_load:
    cols_to_load.append('fiber_100g')
if 'alcohol_100g' not in cols_to_load:
    cols_to_load.append('alcohol_100g')
    
# On charge uniquement les colonnes utiles
df_original = pd.read_csv(filename, sep='\t', usecols=cols_to_load)
df_original.shape

E:\Users\Windows\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(1555491, 25)

In [4]:
pd.options.display.max_rows = None  # option permettant d'afficher toutes les lignes
inds_nan_rate

code                                          0.000000
url                                           0.000000
creator                                       0.000003
created_t                                     0.000000
created_datetime                              0.000000
last_modified_t                               0.000000
last_modified_datetime                        0.000000
product_name                                  0.051377
generic_name                                  0.930710
quantity                                      0.725436
packaging                                     0.840568
packaging_tags                                0.840577
packaging_text                                0.999308
brands                                        0.450540
brands_tags                                   0.450576
categories                                    0.492691
categories_tags                               0.492692
categories_en                                 0.492692
origins   

In [5]:
# Nombre de colonnes supprimées
print(f"{len(inds_nan_rate)-df_original.shape[1]} colonnes ont été supprimées.")

158 colonnes ont été supprimées.


In [6]:
pd.reset_option('display.max_rows')

La série *inds_nan_rate* donne le taux de valeurs manquantes pour chaque colonne. On voit ainsi que les métadonnées de chaque produit sont quasi-systématiquement renseignées mais les données qui rentrent trop dans le détail de la composition des produits sont quasiment inexistantes.
On peut regarder à quoi ressemble notre dataframe de départ:

In [7]:
pd.options.display.max_columns = None
df_original.head()

,code,url,product_name,brands,countries_en,additives_n,ingredients_from_palm_oil_n,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,main_category_en,image_small_url,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,Vitória crackers,NaN,France,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,https://static.openfoodfacts.org/images/produc...,375.0,1569.0,7.0,3.08,70.1,15.0,NaN,7.8,1.4,0.56,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,Cacao,NaN,France,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000000003327986,http://world-en.openfoodfacts.org/product/0000...,Filetes de pollo empanado,NaN,Spain,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000100,http://world-en.openfoodfacts.org/product/0000...,moutarde au moût de raisin,courte paille,France,0.0,0.0,18.0,d,NaN,Fat and sauces,Dressings and sauces,Mustards,https://static.openfoodfacts.org/images/produc...,NaN,936.0,8.2,2.20,29.0,22.0,0.0,5.1,4.6,1.84,NaN
4,00000000001111111111,http://world-en.openfoodfacts.org/product/0000...,Sfiudwx,Watt,France,NaN,NaN,NaN,NaN,NaN,unknown,unknown,fr:xsf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Provenance des produits
On peut s'intéresser aux sources des données en regardant quels pays sont les plus représentés:

In [8]:
countries = df_original['countries_en'].value_counts().head(10).to_frame()
countries

,countries_en
France,677031
United States,333949
Spain,177376
Belgium,42890
Germany,41057
Switzerland,39327
United Kingdom,28595
Canada,20252
Italy,15045
"France,Germany",11064


La base de données étant à l'origine une initiative française, on retrouve en première position la France, suivie des Etats-Unis. On peut donc s'attendre à ce que la majorité des produits aient des noms français ou anglais.

In [9]:
brands= df_original['brands'].value_counts().head(10).to_frame()
brands

,brands
Carrefour,13412
Auchan,11014
U,6057
Bonarea,5638
Delhaize,4873
Hacendado,4682
Casino,4435
Nestlé,4354
Leader Price,4126
Cora,3385


En effectuant le même classement sur les marques on retrouve bien des marques françaises en majorité.

### Nettoyage du jeu de données
#### Présence de doublons
Vérifions en premier lieu la présence de doublons. Chaque produit est censé avoir un unique code qui lui est dédié. Il est important de vérifier que cette propriété soit vraie car dans la suite nous identifierons un produit par son code.

In [10]:
df_clean = df_original.copy()

df_clean['code'] = df_clean['code'].astype(str)
gb = df_clean.groupby(by='code')
doublons = (gb.size()-1).sum()
doublons

715

Le dataset contient 715 doublons, nous les supprimons à l'aide la méthode *drop_duplicates*.

In [11]:
df_clean.drop_duplicates(subset='code', inplace=True)

### Suppression des valeurs aberrantes
La base de données est remplie par des contributeurs divers sur la base du volontariat. Elle comporte par conséquent beaucoup d'erreurs, soient-elles des fautes de frappe (un zéro en trop, une virgule au mauvais endroit), ou une donnée renseignée dans le mauvais champ, etc...
La majorité des données étant spécifiée pour 100g de produit, on peut facilement éliminer les données aberrantes lorsqu'elles 
ne respectent pas les bilans massique ou énergétique:

In [12]:
# On supprime les valeurs qui ne sont pas comprises entre 0 et 100
liste_100g = df_original.iloc[:, -10:].columns.tolist()
for col in liste_100g:
    if col != 'energy_100g':
        df_clean = df_clean[(df_clean[col] <= 100) & (df_clean[col] >= 0) | df_clean[col].isnull()]

In [26]:
# On supprime les lignes qui n'ont aucune info nutritionnelle
df_clean.dropna(subset=liste_100g, how='all', inplace=True)

# On supprime les lignes qui ne sont renseignées qu'avec des zéros: il s'agit soit soit d'erreurs, soit d'eau
df_clean = df_clean[df_clean[liste_100g].sum(axis=1) > 0]

# Les sucres ne peuvent dépasser les glucides (mais on garde la ligne lorsqu'une des deux colonnes n'est pas renseignée)
df_clean = df_clean[(df_clean['sugars_100g'] <= df_clean['carbohydrates_100g']) | df_clean['sugars_100g'].isnull() | df_clean['carbohydrates_100g'].isnull()]

# De même les graisses saturées ne peuvent dépasser les lipides
df_clean = df_clean[(df_clean['saturated-fat_100g'] <= df_clean['fat_100g']) | df_clean['saturated-fat_100g'].isnull() | df_clean['fat_100g'].isnull()]

Les macronutriments (lipides, glucides et protéines) ainsi que l'alcool représentent les sources d'énergie des produits. Un bilan sur ces  4 variables nous permet de repérer des valeurs aberrantes car leur total ne peut excéder 100%.

In [24]:
# Le total de macronutriments (+alcool) ne peut excéder 100g
macros = ['fat_100g', 'carbohydrates_100g', 'proteins_100g', 'alcohol_100g']
df_clean['somme_macro'] = df_clean[macros].sum(axis=1)
df_clean = df_clean[df_clean['somme_macro'] <= 100]

# On peut repérer de la même façon les valeurs aberrantes sur la colonne "fibres":
df_clean['somme2'] = df_clean[['sugars_100g', 'fiber_100g', 'fat_100g', 'proteins_100g', 'alcohol_100g', 'salt_100g']].sum(axis=1)
df_clean = df_clean[df_clean['somme2'] <= 100]

Les colonnes **energy-kcal_100g** et **energy_100g** représentent la même chose, la valeur énergétique pour 100g de produit, exprimées respectivement en kcal et en kJ.
En regardant plus en détail les données on se rend compte que la colonne exprimée en Joules est sytématiquement plus remplie. 
Afin de garder l'unité kcal, plus courante en nutrition, on remplit lorsque c'est possible la valeur en kcal en convertissant à partir de la valeur en Joules:

In [15]:
# On remplit les valeurs manquantes de la colonne kcal en convertissant à partir de l'énergie exprimée en Joules
df_clean['energy-kcal_100g'].fillna(df_clean['energy_100g']/4.186, inplace=True)

On peut également recalculer l'énergie de 100g de produit dans la variable **calcul_energie** à partir des quantités de macronutriments renseignées. Si l'énergie calculée de cette façon diffère significativement de la valeur déjà renseignée dans **energy-kcal_100g** il y a erreur dans une des données, ce qui nous permet de supprimer la ligne.

In [16]:
# Pour la suite des calculs les valeurs manquantes sur les macronutriments (et l'alcool) doivent être renseignées
for col in macros:
    df_clean[col].fillna(0, inplace=True)
    
# La valeur de l'énergie doit correspondre au total des sources d'énergie
df_clean['calcul_energie'] = df_clean['fat_100g'] * 9 + (df_clean['carbohydrates_100g'] + df_clean['proteins_100g']) * 4 + df_clean['alcohol_100g'] * 7

def diff_perc(series1, series2):
    diff = abs(2*(series1 -series2)/(series1+series2))
    return diff

# On choisit une marge d'erreur acceptable de 10%
df_clean = df_clean[(diff_perc(df_clean['energy-kcal_100g'], df_clean['calcul_energie']) < 0.1) | df_clean['energy-kcal_100g'].isnull()]

# Moyenne de la différence entre l'énergie recalculée et celle indiquée sur le produit
mean = diff_perc(df_clean['energy-kcal_100g'], df_clean['calcul_energie']).mean() 
mean

0.02319765075299286

L'énergie recalculée de cette façon donne un écart de seulement 2,3% en moyenne avec la valeur renseignée sur le produit, ce qui en fait une approximation très correcte.

La variable catégorielle *pnns_groups_1* contient quelques catégories écrites sous un format différent, on les réécrit correctement:

In [17]:
# Liste des catégories
liste_grp1 = df_clean['pnns_groups_1'].value_counts()
liste_grp1

unknown                    527319
Sugary snacks              110026
Milk and dairy products     70619
Fish Meat Eggs              63812
Cereals and potatoes        61382
Fat and sauces              44236
Composite foods             42550
Beverages                   38864
Salty snacks                24876
Fruits and vegetables       23535
sugary-snacks                3852
fruits-and-vegetables        3178
cereals-and-potatoes           36
salty-snacks                    2
Name: pnns_groups_1, dtype: int64

In [18]:
# Fonction permettant de remplacer les valeurs d'une liste par celles d'une autre liste
def remplacer(df, liste_orig, liste_cible):
    for i in range(len(liste_orig)):
        df.replace(to_replace = liste_orig[i], value = liste_cible[i], inplace = True) 

In [19]:
# On renomme les catégories qui avaient une syntaxe différente
liste_orig = ['sugary-snacks', 'fruits-and-vegetables', 'cereals-and-potatoes', 'salty-snacks']
liste_cible = ['Sugary snacks', 'Fruits and vegetables', 'Cereals and potatoes', 'Salty snacks']
remplacer(df_clean, liste_orig, liste_cible)

In [20]:
# On vérifie que les catégories sont correctement raccordées
liste_grp1 = df_clean['pnns_groups_1'].value_counts()
liste_grp1

unknown                    527319
Sugary snacks              113878
Milk and dairy products     70619
Fish Meat Eggs              63812
Cereals and potatoes        61418
Fat and sauces              44236
Composite foods             42550
Beverages                   38864
Fruits and vegetables       26713
Salty snacks                24878
Name: pnns_groups_1, dtype: int64

A ce stade, notre dataset est cleané et prêt à être analysé. Il n'est pas exempt d'erreurs, mais toutes les valeurs aberrantes qui peuvent être repérées de façon automatique ont été supprimées.

In [21]:
df_clean

,code,url,product_name,brands,countries_en,additives_n,ingredients_from_palm_oil_n,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,main_category_en,image_small_url,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,alcohol_100g,somme_macro,somme2,calcul_energie
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,Vitória crackers,NaN,France,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,https://static.openfoodfacts.org/images/produc...,375.000000,1569.0,7.0,3.08,70.1,15.0,NaN,7.8,1.400,0.5600,0.0,84.9,29.8,374.6
3,0000000000100,http://world-en.openfoodfacts.org/product/0000...,moutarde au moût de raisin,courte paille,France,0.0,0.0,18.0,d,NaN,Fat and sauces,Dressings and sauces,Mustards,https://static.openfoodfacts.org/images/produc...,223.602484,936.0,8.2,2.20,29.0,22.0,0.0,5.1,4.600,1.8400,0.0,42.3,35.3,210.2
5,0000000000123,http://world-en.openfoodfacts.org/product/0000...,Sauce Sweety chili 0%,NaN,France,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,https://static.openfoodfacts.org/images/produc...,21.000000,88.0,0.0,0.00,4.8,0.4,NaN,0.2,2.040,0.8160,0.0,5.0,0.6,20.0
14,0000000000970,http://world-en.openfoodfacts.org/product/0000...,Fromage blanc aux myrtilles,NaN,France,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,https://static.openfoodfacts.org/images/produc...,129.000000,540.0,4.9,3.10,16.3,16.3,NaN,4.4,0.250,0.1000,0.0,25.6,25.6,126.9
17,0000000001137,http://world-en.openfoodfacts.org/product/0000...,Baguette parisien,NaN,France,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,https://static.openfoodfacts.org/images/produc...,222.000000,929.0,3.3,2.10,38.4,1.8,NaN,11.7,0.678,0.2712,0.0,53.4,16.8,230.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555478,9999991149090,http://world-en.openfoodfacts.org/product/9999...,Riz parfumé,King Elephant,France,NaN,NaN,0.0,b,NaN,Cereals and potatoes,Cereals,Aromatic rices,https://static.openfoodfacts.org/images/produc...,351.000000,1469.0,0.5,0.00,80.0,0.0,NaN,7.0,0.000,0.0000,0.0,87.5,7.5,352.5
1555480,99999988,http://world-en.openfoodfacts.org/product/9999...,Boules,NaN,France,NaN,NaN,NaN,NaN,NaN,unknown,unknown,NaN,https://static.openfoodfacts.org/images/produc...,372.000000,1556.0,11.0,1.30,58.0,54.0,NaN,6.3,0.020,0.0080,0.0,75.3,71.3,356.2
1555482,9999999004360,http://world-en.openfoodfacts.org/product/9999...,Minis beignets,NaN,France,NaN,NaN,15.0,d,NaN,unknown,unknown,Sweet Fritters,https://static.openfoodfacts.org/images/produc...,333.000000,1393.0,20.2,6.00,30.9,10.4,NaN,7.4,1.100,0.4400,0.0,58.5,38.0,335.0
1555483,9999999175305,http://world-en.openfoodfacts.org/product/9999...,Erdbeerkuchen 1019g tiefgefroren,Coppenrath & Wiese,Germany,2.0,0.0,12.0,d,4.0,Sugary snacks,Biscuits and cakes,de:tiefkühl-kuchen,https://static.openfoodfacts.org/images/produc...,220.974677,925.0,7.6,4.80,35.0,24.0,NaN,2.6,0.280,0.1120,0.0,45.2,34.2,218.8


In [27]:
len(df_original) - len(df_clean)

537301

Environ 500 000 erreurs ont ainsi été supprimées, soit près d'un tiers du jeu de données initial.

On peut sauvegarder le dataframe cleané au format csv à l'aide de la méthode *.to_csv()* (supprimer le # pour reproduire l'action):

In [28]:
# df_clean.to_csv("data_clean.csv", index = False)